In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# Original Train data
Or_data=pd.read_csv('train.csv')


# Taking average over time stamps t0-t6
Avg_data=pd.DataFrame()
i=0

while i <= 384:
    df=Or_data.iloc[:,i+1:i+8].mean(axis=1)
#     if i < 8:
#         n=0
#     else: 
#         n=(i-1)/7
    
    Avg_data[i/7]=df
    #print(i, df_train.columns[i])
    i +=7
Avg_data['y2'] = Or_data['y2']
Avg_data['Id'] = Or_data['Id']

Avg_data.head()

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,y2,Id
0,760.244080,0.937269,1.0,22.056471,6.728571,748.549404,789.804275,811.182847,4.4,0.269143,...,0.857143,700.196065,119.060403,14.758329,878.637498,646.594635,89.693307,1.892407,0.72,706
1,757.366380,1.379567,1.0,26.895488,6.500000,749.596987,845.599976,840.799988,4.5,0.279756,...,0.928571,547.953130,132.105129,19.044833,886.672018,712.281086,74.327068,1.913206,0.62,239
2,757.052327,0.864291,1.0,22.271551,6.900000,820.260866,793.147153,815.468561,4.4,0.209458,...,0.571429,547.210846,109.675539,13.939582,861.906494,676.498573,87.959533,2.052945,0.57,613
3,760.571769,0.808450,1.0,28.319341,6.900000,753.283047,793.147153,815.468561,4.4,0.181063,...,0.857143,630.162187,137.102413,14.898321,871.835244,653.640959,88.314801,1.940227,1.01,588
4,768.801759,0.822851,1.0,19.236735,6.814286,760.840050,775.918551,766.868551,4.4,0.290602,...,0.714286,497.926779,91.554997,14.059461,855.777351,640.448159,87.042743,2.248662,-0.42,780


# Now converting each row to normal distribution

In [107]:
Con_data=Avg_data.iloc[:,0:-1]
from sklearn.preprocessing import StandardScaler

data_trans = StandardScaler().fit_transform(Con_data)

Con_data.to_csv('dat.csv')
pd.DataFrame(data_trans).to_csv("moddat.csv")
# Making Data-frame of Transformed data
df2=pd.DataFrame(data_trans,index=[i for i in range(data_trans.shape[0])],columns=['x_'+str(i) for i in range(data_trans.shape[1]) ])

# To rename the column x_55 to y2
df2.rename(columns = {'x_55': 'y2'}, inplace = True)

# preparing the dataset for taking covariance matrix

In [26]:
df3=df2.iloc[:,0:-1]

# Calculating the covMatrix
data = np.array(df3)
#data=np.transpose(data)
covMatrix = np.cov(data,rowvar=False,bias=False)
#covMatrix.shape
#print (covMatrix) 
#inva_covMat=np.linalg.inv(covMatrix) 


# Diagonalize the matrix
use svd technique

In [103]:
u, s, vh = np.linalg.svd(data, full_matrices=True)

new_data=np.dot(np.transpose(u),data)
new_data.shape


Mod_covMatrix = np.cov(new_data,rowvar=False,bias=False)
print(Mod_covMatrix)
# print(np.linalg.det(Mod_covMatrix))
# inva_covMat=np.linalg.inv(Mod_covMatrix)
B = np.linalg.pinv(Mod_covMatrix)
B
Mod_covMatrix = np.where(Mod_covMatrix < 0.50, 0, Mod_covMatrix)
print(Mod_covMatrix)

[[ 1.00098952e+00 -1.62216162e-01 -3.95108158e-02 ... -2.70187441e-01
  -1.34911372e-01 -3.30766275e-01]
 [-1.62216162e-01  1.00105212e+00  5.46463972e-02 ...  1.15109098e-01
   2.63140462e-01  4.18959406e-01]
 [-3.95108158e-02  5.46463972e-02  1.00056826e+00 ...  6.86369289e-02
  -2.64786127e-02 -7.35975740e-04]
 ...
 [-2.70187441e-01  1.15109098e-01  6.86369289e-02 ...  1.00084252e+00
   3.09975255e-01  5.25972892e-01]
 [-1.34911372e-01  2.63140462e-01 -2.64786127e-02 ...  3.09975255e-01
   1.00070369e+00  5.10546580e-01]
 [-3.30766275e-01  4.18959406e-01 -7.35975740e-04 ...  5.25972892e-01
   5.10546580e-01  1.00062521e+00]]
[[1.00098952 0.         0.         ... 0.         0.         0.        ]
 [0.         1.00105212 0.         ... 0.         0.         0.        ]
 [0.         0.         1.00056826 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.00084252 0.         0.52597289]
 [0.         0.         0.         ... 0.         1.00070369 0.5105

In [89]:
# dat=np.transpose(data)

# Y=np.dot(data,covMatrix,dat)

# u, s, vh = np.linalg.svd(covMatrix, full_matrices=True)
da=Avg_data.iloc[:,0:-2]
da.head()

u, s, vh = np.linalg.svd(data, full_matrices=True)
#print(np.dot(np.transpose(vh),vh))
#u.shape[0]
# import numpy as np
Sigma = np.zeros((u.shape[0], vh.shape[0]))
Sigma.shape
for i in range(vh.shape[0]):
    Sigma[i, i] = s[i]
da1=np.dot(np.dot(u,Sigma),np.transpose(vh))
# print(da1)
Sigma

# print('u dimension',u.shape)
# print('v dimension',vh.shape)
# print('s dimension',s.shape)
# Tr_dat=np.dot(vh,np.transpose(da))

# tr_dat=np.transpose(Tr_dat)
# tr_dat.shape

# Tr_covMat = np.cov(tr_dat,rowvar=False,bias=False)
# print(Tr_covMat)

array([[91.15229893,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 69.01885527,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 62.36293377, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [40]:
# Making Data-frame of Transformed data
df4=pd.DataFrame(u,index=[i for i in range(u.shape[0])],columns=['x_'+str(i) for i in range(u.shape[1]) ])

In [44]:
df4.shape

(794, 794)

In [49]:
from sklearn.decomposition import PCA 

pca = PCA(0.9999999)

pca.fit(data)

pca.n_components_ 

X_trans_new = pca.transform(data)
X_trans_new.shape

(794, 54)

In [55]:
covMat = np.cov(X_trans_new,rowvar=False,bias=False)
inva_covMat=np.linalg.inv(covMat)
inva_covMat.shape
dat=np.transpose(X_trans_new)
dat.shape

(54, 794)

In [57]:
Y=np.dot(np.dot(X_trans_new,inva_covMat),dat)

In [59]:
np.count_nonzero(covMat - np.diag(np.diagonal(covMat)))


2858